
# Compute Spectra Workflow

### Welcome to Optical Property Suite! Built on QuantumEspresso v6.8 and Hublib via SimTool v0.3.3
#### To begin computationally exploring the optical properties of a material of interest simply:
1. Identify material system of interest. Options:
  - Choose from an online crystals database
  - Write or enter your own crystal file(s)
2. Specify simulation parameters
3. Run simulation
4. View standard summary of results

To more thoroughly explore the data aggregated by the users of this tool on nanoHUB, go the the *Analyze Spectra Workflow* app.

In [ ]:
%load_ext autoreload
%autoreload 2

/apps/share64/debian7/anaconda/anaconda-6/lib/python3.7/site-packages/xarray/core/merge.py:10: FutureWarning: The Panel class is removed from pandas. Accessing it from the top-level namespace will also be removed in the next version
  PANDAS_TYPES = (pd.Series, pd.DataFrame, pd.Panel)


In [1]:
#user interface utilities 
import pandas as pd

from simtool import findInstalledSimToolNotebooks,searchForSimTool
from simtool import getSimToolInputs,getSimToolOutputs,Run

from hublib.ui import Tab, Form
from frontend import *

/apps/share64/debian10/anaconda/anaconda-7/lib/python3.8/site-packages/scrapbook/__init__.py:8: FutureWarning: 'nteract-scrapbook' package has been renamed to `scrapbook`. No new releases are going out for this old package name.
  warnings.warn("'nteract-scrapbook' package has been renamed to `scrapbook`. No new releases are going out for this old package name.", FutureWarning)


<IPython.core.display.Javascript object>

 ### Authenticate to use remote materials databases 
 You will be automatically authenticated to use a database if you have a corresponding keyfile in your nanohub home directory.

In [2]:
AU = Authenticate() #isolate instantiation for the renewal mechanism to function

In [3]:
mpForm = Form([AU.mpkey()], name="Materials Project")

AuthTabs = Tab([mpForm], desc="Sign into given database servers to use their records")
Form([AuthTabs, AU.renew_button], name="Authenticate")

Group(children=(HTML(value="<p   style='background-color: #DCDCDC; font-size: 150%; padding: 5px'>Authenticate…

### Perform Remote Queries and Switch Between Them. Optional.
You must be authenticate into each databases you wish to use.

In [4]:
qp = QueryPanel()
Form([qp.toggles, qp.progressout], name="Pick Data Source")

Group(children=(HTML(value="<p   style='background-color: #DCDCDC; font-size: 150%; padding: 5px'>Pick Data So…

In [8]:
ss = InputSuite(qp.Q)
remoteform = Form([ss.remote_menu],
                  name="Retrieve",
                  desc="Search for structures by ID in the toggled database")
textform = Form([ss.copybox, ss.plotbutton],
                name="Write",
                desc='accepts POSCAR, cif, vasprun.xml, CSSR, Netcdf and pymatgen’s JSON-serialized structures')
fileform = Form([ss.upload_button, ss.files_menu],
                name="Upload",
                desc='accepts multiple of cif, poscar, cssr, json, yaml, xsf, mcsqs')
InTabs = Tab([remoteform, textform, fileform], desc="Input a Structure to use in Simulations")
Form([InTabs, ss.plotout], name="Select or Provide a Semiconductor to Simulate")

Group(children=(HTML(value="<p   style='background-color: #DCDCDC; font-size: 150%; padding: 5px'>Select or Pr…

In [15]:
ss.struct

Structure Summary
Lattice
    abc : 4.110279 4.110279 4.110279
 angles : 90.0 90.0 90.0
 volume : 69.44067064750126
      A : 4.110279 0.0 0.0
      B : 0.0 4.110279 0.0
      C : 0.0 0.0 4.110279
PeriodicSite: Zr (0.0000, 0.0000, 0.0000) [0.0000, 0.0000, 0.0000]
PeriodicSite: B (0.8179, 2.0551, 2.0551) [0.1990, 0.5000, 0.5000]
PeriodicSite: B (2.0551, 2.0551, 3.2923) [0.5000, 0.5000, 0.8010]
PeriodicSite: B (2.0551, 2.0551, 0.8179) [0.5000, 0.5000, 0.1990]
PeriodicSite: B (2.0551, 0.8179, 2.0551) [0.5000, 0.1990, 0.5000]
PeriodicSite: B (2.0551, 3.2923, 2.0551) [0.5000, 0.8010, 0.5000]
PeriodicSite: B (3.2923, 2.0551, 2.0551) [0.8010, 0.5000, 0.5000]

# Perform a series of simulations to obtain optical spectra

### Find relax_sim simtool notebook and confirm

In [9]:
#simToolName = "670raman"
simToolName = "relax_sim"
simToolLocation = searchForSimTool(simToolName)
for key in simToolLocation.keys():
    print(f"{key} = {simToolLocation[key]}")

notebookPath = /home/nanohub/nykiel.4/opt_prop/opt_prop_suite/simtool/relax_sim.ipynb
simToolName = relax_sim
simToolRevision = None
published = False


In [ ]:
ecutwfc_list = [50]
ecutrho_list = []
kpoints_list = [4]
struct_dict_list = []
results_list = []

inputs['pp_class'].value = 'PAW'
inputs['struct_dict'].value = struct_dict

for ecutwfc in ecutwfc_list:
    for kpoints in kpoints_list:
        inputs['ecutwfc'].value = ecutwfc
        inputs['kpoints'].value = kpoints
        print("running sim with ",ecutwfc," and ",kpoints)
        r = Run(simToolLocation,inputs)
        results_list.append(r.getResultSummary())

In [ ]:
# get list of available pseudopotential files

pp_list = []

for filename in os.listdir("./simtool/pseudo/"):
    f = os.path.join("./simtool/pseudo/", filename)
    # get a list of all the PPs -- is this best instatiated here or globally?
    # if instanced here, the user could probably pass their own PPs to the constructor as well
    if os.path.isfile(f):
        pp_list.append(filename)

# filter by selected compound compositions
elements = np.unique([''.join([i for i in str(i.species) if i.isalpha()]) for i in struct.sites])
filtered_pp_list = [pp for pp in pp_list for e in elements if e in pp]

# make widgets for sim2l parameters

log = widgets.Select(
    options=["DEBUG", "INFO", "WARNING", "ERROR", "CRITICAL"],
    value='DEBUG',
    # rows=10,
    description='Log Level:',
)

walltime = widgets.Text(
    value='01:00:00',
    placeholder='walltime',
    description='walltime:',
    disabled=False
)
numnodes = widgets.IntText(
    value=8,
    placeholder='nodes',
    description='nodes:',
    disabled=False
) 
button = widgets.Button(
    description='run simtool',
    disabled=False,
    button_style='', # 'success', 'info', 'warning', 'danger' or ''
    tooltip='run to submit qe simtool'
)
storage = widgets.Select(
    options=["none", "nowf", "low", "medium"],
    value="none",
    description='wfn + dat files to save:',
)
ecutwfc = widgets.BoundedFloatText(
    value=50,
    min=50,
    max=400,
    step=10,
    description='ecutwfc:',
    disabled=False
)
kpoints = widgets.BoundedFloatText(
    value=6,
    min=1,
    max=20,
    step=1,
    description='kpoints:',
    disabled=False
)
ecutrho = widgets.BoundedFloatText(
    value=200,
    min=200,
    max=1600,
    step=40,
    description='ecutrho:',
    disabled=False
)
smearing = widgets.Select(
    options=['smearing','fixed'],
    value='fixed',
    rows = 2,
    description='smearing:',
    disabled=False
)
pp_menu1 = widgets.Combobox(
    placeholder="choose a pseudopotential",
    options=filtered_pp_list,
    description='pseudo 1:',
    disabled=False
)
pp_menu2 = widgets.Combobox(
    placeholder="choose a pseudopotential",
    options=filtered_pp_list,
    description='pseudo 2:',
    disabled=False
)

output = widgets.Output()

def runSim2l():
    inputs['loglevel'].value = log.value
    inputs['walltime'].value = walltime.value
    inputs['numnodes'].value = numnodes.value
    inputs['storage'].value = storage.value
    inputs['ecutwfc'].value = ecutwfc.value
    inputs['ecutrho'].value = ecutrho.value
    inputs['kpoints'].value = kpoints.value
    inputs['pps'].value = [pp_menu1.value, pp_menu2.value]
    inputs['smearing'].value = smearing.value
    inputs['struct_dict'].value = struct_dict

def on_button_clicked(b):
    with output:
        runSim2l()
        r = Run(simToolLocation,inputs)
        results = r.getResultSummary()
        print(r.read('spectra'))
    return results
        
        
results = button.on_click(on_button_clicked)

simulation = widgets.VBox([ecutrho, ecutwfc, kpoints, smearing, pp_menu1, pp_menu2])
run_details = widgets.VBox([walltime, numnodes, log])

accordion = widgets.VBox([widgets.Accordion(children=[simulation,run_details]),button,output])
display(accordion)

### Show User Predetermined Outputs and their Explainations

In [ ]:
outputs = getSimToolOutputs(simToolLocation)
outputs

### Run simtool to obtain Predicted Raman Tensor and Spectrum Graph

In [ ]:
r.getResultSummary()

In [ ]:
results.read('spectra')

In [ ]:
r.read('spectra')

In [ ]:
print(r.read('logreport'))

In [ ]:
#check inputs
r.input_dict

In [ ]:
#find output location
print(r.outdir)